本节我们介绍批量归一化（batch normalization）层，它能让较深的神经网络的训练变得更加容易 [1]。在 “[实战Kaggle比赛：预测房价](http://zh.d2l.ai/chapter_deep-learning-basics/kaggle-house-price.html)” 一节里，我们对输入数据做了标准化处理：处理后的任意一个特征在数据集中所有样本上的均值为0、标准差为1。**标准化处理输入数据使各个特征的分布相近：这往往更容易训练出有效的模型**。

通常来说，数据标准化预处理对于浅层模型就足够有效了。随着模型训练的进行，当每层中参数更新时，**靠近输出层的输出较难出现剧烈变化**。**但对深层神经网络来说**，即使输入数据已做标准化，**训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化**。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，**批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。**批量归一化和下一节将要介绍的**残差网络**为训练和设计深度模型提供了两类重要思路。

## 5.10.1 批量归一化层

对全连接层和卷积层做批量归一化的方法稍有不同。下面我们将分别介绍这两种情况下的批量归一化。

### 5.10.1.1. 对全连接层做批量归一化

我们先考虑如何对全连接层做批量归一化。通常，我们将批量归一化层置于全连接层中的仿射变换和激活函数之间。设全连接层的输入为$\boldsymbol{u}$，权重参数和偏差参数分别为$\boldsymbol{W}$和$\boldsymbol{b}$，，激活函数为ϕ。设批量归一化的运算符为$\boldsymbol{BN}$。那么，使用批量归一化的全连接层的输出为：
$$\phi(\text{BN}(\boldsymbol{x})),$$
其中批量归一化输入由$\boldsymbol{x}$仿射变换
$$\boldsymbol{x} = \boldsymbol{W\boldsymbol{u} + \boldsymbol{b}}$$
得到。考虑一个由m个样本组成的小批量，仿射变换的输出为一个新的小批量$\mathcal{B} = \{\boldsymbol{x}^{(1)}, \ldots, \boldsymbol{x}^{(m)} \}$。它们正是批量归一化层的输入。对于小批量$\mathcal{B}$中任意样本$\boldsymbol{x}^{(i)} \in \mathbb{R}^d, 1 \leq i \leq m$，批量归一化层的输出同样是$\boldsymbol{d}$维向量
$$\boldsymbol{y}^{(i)} = \text{BN}(\boldsymbol{x}^{(i)}),$$
并由以下几步求得。首先，对小批量$\mathcal{B}$求均值和方差：
$$\boldsymbol{\mu}_\mathcal{B} \leftarrow \frac{1}{m}\sum_{i = 1}^{m} \boldsymbol{x}^{(i)},$$
$$\boldsymbol{\sigma}_\mathcal{B}^2 \leftarrow \frac{1}{m} \sum_{i=1}^{m}(\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B})^2,$$
其中的平方计算是按元素求平方。接下来，使用按元素开方和按元素除法对$\boldsymbol{x}^{(i)}$标准化：
$$\hat{\boldsymbol{x}}^{(i)} \leftarrow \frac{\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B}}{\sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}},$$
这里$\epsilon > 0$是一个很小的常数，保证分母大于0。在上面标准化的基础上，批量归一化层引入了两个可以学习的模型参数，拉伸（scale）参数$\boldsymbol{\gamma}$和偏移（shift）参数$\boldsymbol{\beta}$。这两个参数和$\boldsymbol{x}^{(i)}$形状相同，皆为d维向量。它们与$\hat{\boldsymbol{x}}^{(i)}$分别做按元素乘法（符号$\odot$）和加法计算：
$${\boldsymbol{y}}^{(i)} \leftarrow \boldsymbol{\gamma} \odot \hat{\boldsymbol{x}}^{(i)} + \boldsymbol{\beta}.$$
至此，我们得到了$\boldsymbol{x}^{(i)}$的批量归一化的输出$\boldsymbol{y}^{(i)}$。 值得注意的是，可学习的拉伸和偏移参数保留了不对$\boldsymbol{x}^{(i)}$做批量归一化的可能：此时只需学出$\boldsymbol{\gamma} = \sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}$和$\boldsymbol{\beta} = \boldsymbol{\mu}_\mathcal{B}$。我们可以对此这样理解：**如果批量归一化无益，理论上，学出的模型可以不使用批量归一化**。

这一点进步确实好大！

### 5.10.1.2. 对卷积层做批量归一化

**对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前**。如果卷积计算输出多个通道，我们需要对这些通道的输出**分别**做批量归一化，**且每个通道都拥有独立的拉伸和偏移参数，并均为标量**。设小批量中有 m个样本。**在单个通道上**，假设卷积计算输出的高和宽分别为 p和 q 。**我们需要对该通道中 m×p×q个元素同时做批量归一化**。**对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中 m×p×q个元素的均值和方差。**

### 5.10.1.3. 预测时的批量归一化

**使用批量归一化训练时，我们可以将批量大小设得大一点**，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，**我们希望模型对于任意输入都有确定的输出**。因此，**单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差**。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

### 5.10.2. 从零开始实现

下面我们通过numpy中的ndarray来实现批量归一化层。

In [1]:
def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var,eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / np.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2,4)   # X最多4维
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(axis=0)
            var = ((X-mean)**2).mean(axis=0)
        else:
            # 使用二维卷积层的情况，计算通道维度上（axis=1)的均值和方差。这里我们需要保持
            #X的形状以便后面可以做广播运算
            mean = X.mean(axis=(0,2,3), keepdims=True)
            var = ((X-mean)**2).mean(axis=(0,2,3), keepdims=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X-mean) / np.sqrt(var+eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0-momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y  = gamma * X_hat + beta   #拉伸和偏移
    return Y, moving_mean, moving_var

接下来，我们自定义一个BatchNorm层。它**保存**参与求梯度和迭代的拉伸参数gamma和偏移参数beta，同时也**维护**移动平均得到的均值和方差，以便能够在模型预测时被使用。BatchNorm实例所需指定的num_features参数**对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数**。该实例所需指定的num_dims参数对于全连接层和卷积层来说分别为2和4。

In [2]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [3]:
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
class BatchNormalization(tf.keras.layers.Layer):
    def __init__(self, decay=0.9, epsilon=1e-5, **kwargs):
        self.decay = decay   # 这个参数是什么意思？
        self.epsilon = epsilon   # 这个参数是什么意思？
        super(BatchNormalization, self).__init__(**kwargs)  #执行父类中的init方法
    
    def build(self, input_shape):      # 这个函数这里的用法，我看的一愣一愣的
        self.gamma = self.add_weight(name='gamma',
                                     shape=[input_shape[-1], ],
                                     initializer=tf.initializers.ones,
                                     trainable=True)
        self.beta = self.add_weight(name='beta',
                                    shape=[input_shape[-1], ],
                                    initializer=tf.initializers.zeros,
                                    trainable=True)
        self.moving_mean = self.add_weight(name='moving_mean',
                                           shape=[input_shape[-1], ],
                                           initializer=tf.initializers.zeros,
                                           trainable=False)
        self.moving_variance = self.add_weight(name='moving_variance',
                                               shape=[input_shape[-1], ],
                                               initializer=tf.initializers.ones,
                                               trainable=False)
        super(BatchNormalization, self).build(input_shape)   # 激活父类中的 build 方法
        
    def assign_moving_average(self, variable, value):
        """
        variable = variable * decay + value * (1 - decay)
        """
        delta = variable * self.decay + value * (1 - self.decay)   # 这一步公式计算，是什么意思？
        return variable.assign(delta)
    
    '''
    @tf.function 基础使用方法
    在 TensorFlow 2.0 中，推荐使用 @tf.function （而非 1.X 中的 tf.Session ）实现 Graph Execution，
    从而将模型转换为易于部署且高性能的 TensorFlow 图模型。只需要将我们希望以 Graph Execution 模式运行的代码封装在一个函数内，
    并在函数前加上 @tf.function 即可.
    @tf.function 使用静态编译将函数内的代码转换成计算图
    '''
    @tf.function
    def call(self, inputs, training):
        if training:
            batch_mean, batch_variance = tf.nn.moments(inputs, list(range(len(inputs.shape) - 1)))  #tf.nn.moments()用于计算均值和方差
            # list(range(len(inputs.shape) - 1))，该公式常用来求input的维度信息。 需要进行求均值/方差的维度
            mean_update = self.assign_moving_average(self.moving_mean, batch_mean)   #更新均值
            variance_update = self.assign_moving_average(self.moving_variance, batch_variance)   #更新方差
            self.add_update(mean_update)
            self.add_update(variance_update)
            mean, variance = batch_mean, batch_variance
        else:
            mean, variance = self.moving_mean, self.moving_variance
        # tf.nn.batch_normalization，用于最终执行batch normalization的函数
        output = tf.nn.batch_normalization(inputs,
                                           mean=mean,
                                           variance=variance,
                                           offset=self.beta,   #样本偏移(相加一个转化值)
                                           scale=self.gamma,   #缩放
                                           variance_epsilon=self.epsilon)  # 为了避免分母为0，添加的一个极小值
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

### 5.10.2.1. 使用批量归一化层的LeNet

下面我们修改“[卷积神经网络（LeNet）](http://zh.d2l.ai/chapter_convolutional-neural-networks/lenet.html)”这一节介绍的LeNet模型，从而应用批量归一化层。我们**在所有的卷积层或全连接层之后、激活层之前**加入批量归一化层。

In [5]:
net = tf.keras.models.Sequential(
    [tf.keras.layers.Conv2D(filters=6,kernel_size=5),
    BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=16,kernel_size=5),
    BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120),
    BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(84),
    BatchNormalization(),
    tf.keras.layers.Activation('sigmoid'),
    tf.keras.layers.Dense(10,activation='sigmoid')]
)

下面我们训练修改后的模型。

In [6]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

net.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])
history = net.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)
test_scores = net.evaluate(x_test, y_test, verbose=2) #verbose 是什么意思？
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])  

Epoch 1/5
750/750 [==============================] - 11s 15ms/step - loss: 0.4044 - accuracy: 0.9385 - val_loss: 0.2508 - val_accuracy: 0.9244
Epoch 2/5
750/750 [==============================] - 9s 12ms/step - loss: 0.0829 - accuracy: 0.9774 - val_loss: 0.0802 - val_accuracy: 0.9769
Epoch 3/5
750/750 [==============================] - 9s 12ms/step - loss: 0.0601 - accuracy: 0.9820 - val_loss: 0.1116 - val_accuracy: 0.9648
Epoch 4/5
750/750 [==============================] - 9s 12ms/step - loss: 0.0509 - accuracy: 0.9845 - val_loss: 0.3791 - val_accuracy: 0.8741
Epoch 5/5
750/750 [==============================] - 9s 12ms/step - loss: 0.0429 - accuracy: 0.9867 - val_loss: 0.0680 - val_accuracy: 0.9791
313/313 - 1s - loss: 0.0577 - accuracy: 0.9813
Test loss: 0.057664819061756134
Test accuracy: 0.9812999963760376


###  这个准确率提升，简直可怕！！！  太牛逼了！

最后我们查看第一个批量归一化层学习到的拉伸参数gamma和偏移参数beta。

In [7]:
net.get_layer(index=1).gamma,net.get_layer(index=1).beta

(<tf.Variable 'sequential/batch_normalization/gamma:0' shape=(6,) dtype=float32, numpy=
 array([1.033776  , 1.4345142 , 0.9530587 , 1.0516163 , 0.94318914,
        1.2518121 ], dtype=float32)>,
 <tf.Variable 'sequential/batch_normalization/beta:0' shape=(6,) dtype=float32, numpy=
 array([-0.5691058 ,  0.42187476, -0.41754025,  0.06861018, -0.27802566,
        -0.5435616 ], dtype=float32)>)

## 5.10.3 简洁实现

与我们刚刚自己定义的`BatchNorm`类相比，keras中`layers`模块定义的`BatchNorm`类使用起来更加简单。它不需要指定自己定义的`BatchNorm`类中所需的`num_features`和`num_dims`参数值。在keras中，**这些参数值都将通过延后初始化而自动获取**。下面我们用keras实现使用批量归一化的LeNet。

In [8]:
net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Conv2D(filters=6,kernel_size=5))
net.add(tf.keras.layers.BatchNormalization())   # 变化在这里，之前使用的是 BatchNormalization()，自己设计的方法
net.add(tf.keras.layers.Activation('sigmoid'))
net.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
net.add(tf.keras.layers.Conv2D(filters=16,kernel_size=5))
net.add(tf.keras.layers.BatchNormalization())
net.add(tf.keras.layers.Activation('sigmoid'))
net.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(120))
net.add(tf.keras.layers.BatchNormalization())
net.add(tf.keras.layers.Activation('sigmoid'))
net.add(tf.keras.layers.Dense(84))
net.add(tf.keras.layers.BatchNormalization())
net.add(tf.keras.layers.Activation('sigmoid'))
net.add(tf.keras.layers.Dense(10,activation='sigmoid'))

使用同样的超参数进行训练。

In [9]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

net.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])
history = net.fit(x_train, y_train,
                    batch_size=64,
                    epochs=5,
                    validation_split=0.2)
test_scores = net.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Epoch 1/5
750/750 [==============================] - 7s 10ms/step - loss: 0.4025 - accuracy: 0.9413 - val_loss: 0.1721 - val_accuracy: 0.9531
Epoch 2/5
750/750 [==============================] - 7s 10ms/step - loss: 0.0799 - accuracy: 0.9778 - val_loss: 0.2008 - val_accuracy: 0.9387
Epoch 3/5
750/750 [==============================] - 7s 9ms/step - loss: 0.0578 - accuracy: 0.9824 - val_loss: 0.0915 - val_accuracy: 0.9728
Epoch 4/5
750/750 [==============================] - 7s 10ms/step - loss: 0.0465 - accuracy: 0.9857 - val_loss: 0.0847 - val_accuracy: 0.9738
Epoch 5/5
750/750 [==============================] - 7s 10ms/step - loss: 0.0409 - accuracy: 0.9873 - val_loss: 0.2308 - val_accuracy: 0.9284
313/313 - 1s - loss: 0.2137 - accuracy: 0.9339
Test loss: 0.21367329359054565
Test accuracy: 0.933899998664856


## 5.10.4 小结

* **在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络的中间输出，从而使整个神经网络在各层的中间输出的数值更稳定**。
* 对全连接层和卷积层做批量归一化的方法稍有不同。
* 批量归一化层和丢弃层一样，在训练模式和预测模式的计算结果是不一样的。
* keras提供的BatchNorm类使用起来简单、方便。

## 5.10.5 练习

* 能否将批量归一化前的全连接层或卷积层中的偏差参数去掉？为什么？（提示：回忆批量归一化中标准化的定义。）
* 尝试调大学习率。同[“卷积神经网络（LeNet）”](lenet.ipynb)一节中未使用批量归一化的LeNet相比，现在是不是可以使用更大的学习率？
* 尝试将批量归一化层插入LeNet的其他地方，观察并分析结果的变化。
* 尝试一下不学习拉伸参数`gamma`和偏移参数`beta`（构造的时候加入参数`grad_req='null'`来避免计算梯度），观察并分析结果。
* 查看`BatchNorm`类的文档来了解更多使用方法，例如，如何在训练时使用基于全局平均的均值和方差。



### 练习，等待回答

## 5.10.6 参考文献

[1] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.

> 注：本节除了代码之外与原书基本相同，[原书传送门](https://zh.d2l.ai/chapter_convolutional-neural-networks/batch-norm.html)

